In [71]:
import json
from pprint import pprint
import tinydb as tdb

In [72]:
db = tdb.TinyDB('data/tinyDB.json')

In [73]:
osm = db.table('osm')
merimee = db.table('merimee')
wikidata = db.table('wikidata')

In [74]:
dataOSM = osm.all()

In [82]:
dataOSM[0]

{'heritage': '3',
 'heritage:operator': 'mhs',
 'historic': 'memorial',
 'idOSM': 21609292,
 'image': "http://phototheque.1000wallpapers.com/galleries/Nature/p/Pa+%E0+Pl/Parc+de+la+T%EAte+d'Or/Ile+du+souvenir/Monument+aux+morts/large/Monument+aux+morts+5.jpg",
 'mhs:inscription_date': '1982',
 'name': "Monument aux morts de l'île du Souvenir",
 'ref:mhs': 'PA00117982',
 'typeOSM': 'node',
 'wikidata': 'Q17341711',
 'wikipedia': "fr:Monument aux morts de l'île du Souvenir"}

In [99]:
dicoMerimeeIndice = {}
for i, d in enumerate(dataOSM):
    merimeeCode = d['ref:mhs']  # code merimee
    if merimeeCode in dicoMerimeeIndice:
        dicoMerimeeIndice[merimeeCode].append(i)
    else:
        dicoMerimeeIndice[merimeeCode] = [i]
        
listOfDuplicates = [ (refs, indices) for refs, indices in dicoMerimeeIndice.items() if len(indices)>1 ]

listOfDuplicates[:2]

[('PA00092249', [14166, 27098]), ('PA00101383', [10504, 32123])]

In [111]:
len(listOfDuplicates)

488

In [100]:
len( dataOSM )

32246

In [144]:
def getWikiCode(itemUrl):
    items = itemUrl.split('/')
    text = '[%s](%s)'%( items[-1], itemUrl  )
    return text

def formatOSMInfo( info ):
    osmtype = info['typeOSM']
    desc = info['name'] if 'name' in info else '...'
    idOSM = info['idOSM']
    
    osmUrl = 'https://www.openstreetmap.org/%s/%s' % (osmtype, idOSM)
    osmLink = '[%s%s](%s)' % (osmtype, idOSM, osmUrl)
    
    if 'wikidata' in info:
        wikidataUrl = 'https://www.wikidata.org/wiki/%s' % info['wikidata']
        wikidataLink = '- [.wd.](%s)' % wikidataUrl
    else:
        wikidataLink = ''
        
    return '%s %s - %s   ' % ( osmLink, wikidataLink, desc)

page = ''
for merimee, indices in listOfDuplicates:
    page +=  '**%s**:   \n ' % merimee
    page += '\n'.join( [formatOSMInfo(dataOSM[i]) for i in indices] )
    page += '\n\n'

In [145]:
with open('output.txt', 'w') as file:
    file.write( page )

In [ ]:
print(page)